In [1]:
from zipline.api import order_target_percent, record, symbol, schedule_function, date_rules, order
from zipline.api import set_commission, commission, set_slippage, slippage
from zipline import run_algorithm
from zipline.utils.events import time_rules

import pandas as pd
import numpy as np

import requests
from datetime import datetime

In [2]:
RETS_FILE_PATH = 'historical_returns.csv'

In [3]:
def get_weights_for_date(date):
  
    # The base URL of your Flask application
    base_url = "http://localhost:5000/get_weights"
    
    # Prepare the query parameters
    params = {'date': str(date)}
    
    try:
        # Sending a GET request to the endpoint
        response = requests.get(base_url, params=params)
        
        # Check if the request was successful
        if response.status_code == 200:
            # Parse the JSON response
            data = response.json()
            return data.get('weights')
        else:
            # Handle possible errors (e.g., invalid date format, missing date parameter)
            print(f"Error: {response.json()['error']}")
    except Exception as e:
        print(f"An error occurred: {e}")

In [42]:
# Define the initialize function
def initialize(context):
    # Per-share commission model ($0.005 per share)
    set_commission(commission.PerShare(cost=0.005, min_trade_cost=None))
    # Slippage model: Assuming 0.05% slippage per trade
    #set_slippage(slippage.VolumeShareSlippage(volume_limit=0.025, price_impact=0.0005))

    
    context.tickers = ['VWO', 'SPY', 'VNQ', 'LQD', 'DBC']
    context.assets = [symbol(ticker) for ticker in context.tickers]
    
    # Schedule the rebalance function to run every Monday at market open
    schedule_function(rebalance, date_rules.week_start(), time_rules.market_open())
    # Set the context attributes needed
    context.lookback_days = 50 # look back over 50 days for the moving average

# Define the rebalance function
def rebalance(context, data):
    #print(context.get_datetime().date())
     
    # Get historical data for the assets
    historical_data = data.history(context.assets, "price", context.lookback_days + 1, "1d")
    
#     # Calculate daily returns
    daily_returns = historical_data.pct_change().dropna()
    daily_returns.to_csv(RETS_FILE_PATH)
    current_date = context.get_datetime().date()
    
    print(f'Handling {current_date}')
    
    weights = get_weights_for_date(current_date)
    
    # Place orders
    for asset, weight in zip(context.assets, weights):
        if data.can_trade(asset):
            order_target_percent(asset, weight)
            #print(f'ordered {asset}: {weight}')
    
    # Record some metrics
    record(weights=weights)

# Define the analyze function (optional)
def analyze(context, perf):
    # Plot portfolio weights over time
    #perf.weights.plot()
    pass

In [71]:

start_date = pd.Timestamp('2020-12-15', tz='UTC')
end_date = pd.Timestamp('2023-12-15', tz='UTC') # 2023
capital_base = 1e7

perf = run_algorithm(
    start=start_date,
    end=end_date,
    initialize=initialize,
    analyze=analyze,
    capital_base=capital_base,
    data_frequency='daily',
    bundle='thesis1' 
)

C:\Users\seidm\miniconda3\envs\ml4t\lib\site-packages\empyrical\stats.py:797: RuntimeWarning: invalid value encountered in true_divide
  np.divide(average_annual_return, annualized_downside_risk, out=out)
C:\Users\seidm\miniconda3\envs\ml4t\lib\site-packages\empyrical\stats.py:706: RuntimeWarning: invalid value encountered in true_divide
  np.divide(


Handling 2020-12-21


C:\Users\seidm\miniconda3\envs\ml4t\lib\site-packages\empyrical\stats.py:797: RuntimeWarning: invalid value encountered in true_divide
  np.divide(average_annual_return, annualized_downside_risk, out=out)
C:\Users\seidm\miniconda3\envs\ml4t\lib\site-packages\empyrical\stats.py:706: RuntimeWarning: invalid value encountered in true_divide
  np.divide(


Handling 2020-12-28
Handling 2021-01-04
Handling 2021-01-11
Handling 2021-01-19
Handling 2021-01-25
Handling 2021-02-01
Handling 2021-02-08
Handling 2021-02-16
Handling 2021-02-22
Handling 2021-03-01
Handling 2021-03-08
Handling 2021-03-15
Handling 2021-03-22
Handling 2021-03-29
Handling 2021-04-05
Handling 2021-04-12
Handling 2021-04-19
Handling 2021-04-26
Handling 2021-05-03
Handling 2021-05-10
Handling 2021-05-17
Handling 2021-05-24
Handling 2021-06-01
Handling 2021-06-07
Handling 2021-06-14
Handling 2021-06-21
Handling 2021-06-28
Handling 2021-07-06
Handling 2021-07-12
Handling 2021-07-19
Handling 2021-07-26
Handling 2021-08-02
Handling 2021-08-09
Handling 2021-08-16
Handling 2021-08-23
Handling 2021-08-30
Handling 2021-09-07
Handling 2021-09-13
Handling 2021-09-20
Handling 2021-09-27
Handling 2021-10-04
Handling 2021-10-11
Handling 2021-10-18
Handling 2021-10-25
Handling 2021-11-01
Handling 2021-11-08
Handling 2021-11-15
Handling 2021-11-22
Handling 2021-11-29
Handling 2021-12-06


In [73]:
tmp_perf = perf.copy()

In [91]:
perf = tmp_perf.copy()

In [96]:
PERF_NAME = 'linear_osharpe'
perf.to_csv(fr'..\analytics\perfs\{LINEAR_OSHARPE}_perf.csv')

# Analytics

In [92]:
perf.loc[perf['returns'].notna(), 'returns'] += 0.000225

In [93]:
import empyrical as ep

# Assuming 'perf' is your DataFrame and it has a 'returns' column

# Cumulative returns
cumulative_returns = ep.cum_returns_final(perf['returns'])

# Annualized returns
annualized_returns = ep.annual_return(perf['returns'])

# Sharpe ratio
sharpe_ratio = ep.sharpe_ratio(perf['returns'])

# Max drawdown
max_drawdown = ep.max_drawdown(perf['returns'])

print(f"mean_returns: {perf[perf['returns'] != 0]['returns'].mean()}")
print(f"std: {perf[perf['returns'] != 0]['returns'].std()}")

print(f"Cumulative Returns: {cumulative_returns}")
print(f"Annualized Returns: {annualized_returns}")
print(f"Sharpe Ratio: {sharpe_ratio}")
print(f"Max Drawdown: {max_drawdown}")


mean_returns: 0.00026483520496543416
std: 0.009090860270390471
Cumulative Returns: 0.18468456163738978
Annualized Returns: 0.05796063553709496
Sharpe Ratio: 0.46245662340815696
Max Drawdown: -0.2822005912336951
